In [1]:
## information about how to use the script
from FiLMScope.recon_util import (add_sample_entry, get_all_sample_names, get_sample_information,
                                  get_ss_volume_from_dataset)
from FiLMScope.util import play_video
from FiLMScope.calibration import CalibrationInfoManager
from FiLMScope.datasets import FSDataset
import os
from matplotlib import pyplot as plt
import numpy as np

import torch

In [ ]:
# view already saved samples 
get_all_sample_names()

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [4]:
# necessary information about the dataset 
sample_name = "skull_with_tool"
folder = "/media/Friday/Temporary/Clare/FiLMScope_paper_data/skull_with_tool"
image_filename = folder + '/skull_with_tool_20240606_134449_008.nc'
calibration_filename = folder + '/calibration_information'
blank_filename = None 

In [5]:
# this cell will be deleted
# catching up calibration files that didn't include image shape 
manager = CalibrationInfoManager(calibration_filename)
if manager.image_shape is None:
    print("adding image shape")
    downsample = 1
    import xarray as xr 
    dataset = xr.open_dataset(image_filename)
    image_shape = (len(dataset.x), len(dataset.y))
    manager.image_shape = image_shape 
    manager.save_all_info()

In [ ]:
# set up a dataset with just the reference image 
ref_camera = CalibrationInfoManager(calibration_filename).reference_camera
downsample = 4
image_numbers = [ref_camera]
dataset = FSDataset(
    image_filename,
    calibration_filename,
    image_numbers,
    downsample,
    (0, 1, 0, 1),
    enforce_divisible=-1,
    frame_number=-1,
    blank_filename=blank_filename
)

In [ ]:
# pick a crop level that shows the desired FOV
crop = (0, 1024, 0, 768) 
full_image = np.squeeze(dataset.reference_image)
crop_image = full_image[crop[0] : crop[1], crop[2] : crop[3]]

plt.imshow(crop_image)

plt.figure()
plt.imshow(full_image)
plt.colorbar()

In [ ]:
# determine crop values
shape0 = dataset.images.shape[2]
shape1 = dataset.images.shape[3]

crop_values = (
    crop[0] / shape0,
    crop[1] / shape0,
    crop[2] / shape1,
    crop[3] / shape1,
)
print(crop_values)

In [ ]:
dataset = None

# new dataset with the crops
downsample = 7
image_numbers = manager.image_numbers
dataset = FSDataset(
    image_filename,
    calibration_filename,
    image_numbers,
    downsample,
    crop_values,
    enforce_divisible=-1,
    frame_number=-1,
    blank_filename=blank_filename
)

In [ ]:
plt.imshow(dataset.images[40].squeeze())

In [ ]:
# find depth extents
low_plane = -9
high_plane = 4.5

num = 30
depths = np.linspace(low_plane, high_plane, num, endpoint=True)
depth_values = torch.from_numpy(depths).to(torch.float32).cuda()

batch_size = 10
volume = get_ss_volume_from_dataset(dataset, batch_size, depth_values, get_squared=False)
volume = volume.squeeze().cpu().numpy()

In [ ]:
play_video(volume)

In [ ]:
idx = 10
plt.imshow(volume[idx], cmap='gray')
plt.title("{:.2f} mm".format(depths[idx]))

In [23]:
# make the full sample entry
info = {
    "folder": folder,
    "sample_name": sample_name,
    "calibration_filename": calibration_filename,
    "image_filename": image_filename,
    "crop_values": crop_values,
    "depth_range": (low_plane, high_plane),
    "blank_filename": blank_filename,
}

In [ ]:
# and save it!
add_sample_entry(sample_name, info, overwrite=False)